# 🔬 LFM-2.5-Audio Complete Working Implementation

**Based on Official Documentation**: https://github.com/Liquid4All/liquid-audio

This notebook implements **real, working ASR (speech-to-text)** using the official LFM API.

## Key Features:
- ✅ **Real ASR transcription** using `generate_sequential()`
- ✅ **Official API usage** from liquid-audio documentation
- ✅ **Performance metrics** and timing
- ✅ **Quality evaluation** with WER calculation
- ✅ **Multi-turn conversation** examples

In [ ]:
# Cell 1: Setup and Environment Check
import json
import time
from datetime import datetime
from pathlib import Path

import torch
import torchaudio

print("🔧 LFM-2.5-Audio Complete Implementation")
print("=" * 60)

# Environment validation
import sys

print(f"Python: {sys.version.split()[0]}")
print(f"Executable: {sys.executable}")

# Device setup
device = "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Device: {device.upper()}")

# Model config
HF_REPO = "LiquidAI/LFM2.5-Audio-1.5B"
print(f"Model: {HF_REPO}")
print("✅ Setup complete")

In [ ]:
# Cell 2: Import LFM Components (Official API)
from liquid_audio import ChatState, LFM2AudioModel, LFM2AudioProcessor, LFMModality

print("📦 Loading LFM components (Official API)...")

load_start = time.time()

# Load processor and model using official API
processor = LFM2AudioProcessor.from_pretrained(HF_REPO).eval()
model = LFM2AudioModel.from_pretrained(HF_REPO).eval()

load_time = time.time() - load_start

# Move to device if needed
if device != "cpu":
    model = model.to(device)

print(f"✅ Components loaded: {load_time:.2f}s")
print(f"   Processor: {processor.__class__.__name__}")
print(f"   Model: {model.__class__.__name__}")
print(f"   Device: {device.upper()}")
print(f"   Parameters: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
# Cell 3: Load Test Audio
def load_audio_for_lfm(audio_path):
    """Load audio file for LFM processing."""
    # LFM expects audio at any sample rate, but 24kHz is optimal
    waveform, sr = torchaudio.load(str(audio_path))

    # Convert to mono if needed
    if waveform.shape[0] > 1:
        waveform = waveform.mean(dim=0, keepdim=True)

    # Resample to 24kHz for optimal performance
    if sr != 24000:
        resampler = torchaudio.transforms.Resample(sr, 24000)
        waveform = resampler(waveform)
        sr = 24000

    return waveform, sr


# Load test audio
audio_path = Path("data/audio/clean_speech_10s.wav")

if audio_path.exists():
    waveform, sr = load_audio_for_lfm(audio_path)
    print(f"✅ Audio loaded: {audio_path.name}")
    print(f"   Shape: {waveform.shape}")
    print(f"   Sample rate: {sr} Hz")
    print(f"   Duration: {waveform.shape[1] / sr:.1f}s")
else:
    print(f"❌ Audio file not found: {audio_path}")
    # Create dummy audio for testing
    print("Creating dummy audio for testing...")
    waveform = torch.randn(1, 24000 * 5)  # 5 seconds at 24kHz
    sr = 24000

In [ ]:
# Cell 4: ASR Transcription (Official API)
print("🎙️  ASR Transcription (Official API)")
print("=" * 50)

# Create ChatState
chat = ChatState(processor)

# System prompt for ASR (from official docs)
chat.new_turn("system")
chat.add_text("Perform ASR.")
chat.end_turn()

# Add audio input
chat.new_turn("user")
chat.add_audio(waveform, sr)
chat.end_turn()

# Generate transcription
chat.new_turn("assistant")

print("🔍 Transcribing audio...")
start_time = time.time()

transcribed_text = ""
for t in model.generate_sequential(**chat, max_new_tokens=512):
    if t.numel() == 1:  # Text token
        token_text = processor.text.decode(t)
        print(token_text, end="", flush=True)
        transcribed_text += token_text

latency = time.time() - start_time

print("\n\n✅ Transcription complete!")
print(f"   Latency: {latency:.2f}s")
print(f"   Text length: {len(transcribed_text)} characters")

In [ ]:
# Cell 5: Quality Evaluation
def calculate_wer(reference, hypothesis):
    """Calculate Word Error Rate."""
    # Simple WER calculation
    ref_words = reference.lower().split()
    hyp_words = hypothesis.lower().split()

    # Levenshtein distance for word sequences
    m, n = len(ref_words), len(hyp_words)
    dp = [[0] * (n + 1) for _ in range(m + 1)]

    for i in range(m + 1):
        dp[i][0] = i
    for j in range(n + 1):
        dp[0][j] = j

    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if ref_words[i - 1] == hyp_words[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]
            else:
                dp[i][j] = 1 + min(dp[i - 1][j], dp[i][j - 1], dp[i - 1][j - 1])

    return dp[m][n] / max(1, len(ref_words))


# Load ground truth
ground_truth_path = Path("data/text/clean_speech_10s.txt")

if ground_truth_path.exists():
    with open(ground_truth_path) as f:
        ground_truth = f.read().strip()

    print("📊 Quality Evaluation")
    print("=" * 30)
    print(f"Ground Truth: {ground_truth}")
    print(f"Transcription: {transcribed_text}")

    # Calculate WER
    wer = calculate_wer(ground_truth, transcribed_text)

    print("\n📈 Metrics:")
    print(f"   WER: {wer:.3f} ({wer * 100:.1f}%)")
    print(f"   Latency: {latency:.2f}s")
    print(f"   Real-time factor: {latency / (waveform.shape[1] / sr):.2f}x")

    if wer < 0.1:
        print("   ✅ EXCELLENT quality (WER < 10%)")
    elif wer < 0.2:
        print("   ✅ Good quality (WER < 20%)")
    else:
        print("   ⚠️  Needs improvement")
else:
    print("⚠️  Ground truth file not found for comparison")

In [ ]:
# Cell 6: Test TTS (Text-to-Speech)
print("🔊 TTS Test (Text-to-Speech)")
print("=" * 40)

# Create new chat for TTS
chat_tts = ChatState(processor)

# System prompt for TTS with voice selection
chat_tts.new_turn("system")
chat_tts.add_text("Perform TTS. Use the US male voice.")
chat_tts.end_turn()

# Input text
test_text = "Hello, this is a test of the LFM text to speech system."

chat_tts.new_turn("user")
chat_tts.add_text(test_text)
chat_tts.end_turn()

chat_tts.new_turn("assistant")

print(f'Input: "{test_text}"')
print("🔊 Generating speech...")

start_tts = time.time()

audio_out = []
for t in model.generate_sequential(
    **chat_tts, max_new_tokens=512, audio_temperature=0.8, audio_top_k=64
):
    if t.numel() > 1:  # Audio token
        audio_out.append(t)

tts_latency = time.time() - start_tts

# Detokenize audio
if audio_out:
    audio_codes = torch.stack(audio_out[:-1], 1).unsqueeze(0)
    waveform_tts = processor.decode(audio_codes)

    # Save generated audio
    output_path = Path("results/tts_output.wav")
    output_path.parent.mkdir(exist_ok=True)
    torchaudio.save(str(output_path), waveform_tts.cpu(), 24000)

    print("✅ TTS complete!")
    print(f"   Latency: {tts_latency:.2f}s")
    print(f"   Duration: {waveform_tts.shape[1] / 24000:.1f}s")
    print(f"   Saved to: {output_path}")
else:
    print("❌ No audio generated")

In [ ]:
# Cell 7: Multi-turn Conversation Test
print("💬 Multi-turn Conversation Test")
print("=" * 40)

# Create chat with interleaved generation
chat_conv = ChatState(processor)

# System prompt for interleaved generation
chat_conv.new_turn("system")
chat_conv.add_text("Respond with interleaved text and audio.")
chat_conv.end_turn()

# First turn: Audio input
chat_conv.new_turn("user")
chat_conv.add_audio(waveform, sr)  # Use the audio we loaded earlier
chat_conv.end_turn()

chat_conv.new_turn("assistant")

print("🎤 Generating response with interleaved text and audio...")

text_out = []
audio_out = []
modality_out = []

for i, t in enumerate(
    model.generate_interleaved(
        **chat_conv, max_new_tokens=256, audio_temperature=1.0, audio_top_k=4
    )
):
    if t.numel() == 1:  # Text token
        token_text = processor.text.decode(t)
        print(token_text, end="", flush=True)
        text_out.append(t)
        modality_out.append(LFMModality.TEXT)
    else:  # Audio token
        audio_out.append(t)
        modality_out.append(LFMModality.AUDIO_OUT)

    # Safety limit
    if i > 200:
        break

print("\n✅ Conversation turn complete!")
print(f"   Text tokens: {len(text_out)}")
print(f"   Audio tokens: {len(audio_out)}")

# Save audio response if generated
if audio_out:
    audio_codes = torch.stack(audio_out[:-1], 1).unsqueeze(0)
    waveform_response = processor.decode(audio_codes)

    response_path = Path("results/conversation_response.wav")
    response_path.parent.mkdir(exist_ok=True)
    torchaudio.save(str(response_path), waveform_response.cpu(), 24000)
    print(f"   Audio saved to: {response_path}")

In [ ]:
# Cell 8: Performance Metrics
import os

import psutil

print("📊 System Performance Metrics")
print("=" * 40)

# Get process info
process = psutil.Process(os.getpid())
memory_info = process.memory_info()

print("Memory Usage:")
print(f"   RSS: {memory_info.rss / 1e6:.1f} MB")
print(f"   VMS: {memory_info.vms / 1e6:.1f} MB")

print("\n⏱️  Timing Summary:")
print(f"   Model load: {load_time:.2f}s")
print(f"   ASR latency: {latency:.2f}s")
if "tts_latency" in locals():
    print(f"   TTS latency: {tts_latency:.2f}s")

print("\n🎯 Performance Assessment:")
real_time_factor = latency / (waveform.shape[1] / sr)
if real_time_factor < 1.0:
    print("   ✅ REAL-TIME capable (<1.0x)")
elif real_time_factor < 2.0:
    print("   ✅ Near real-time (<2.0x)")
else:
    print("   ⚠️  Not real-time (>2.0x)")

if "wer" in locals():
    print("\n📝 Quality Assessment:")
    if wer < 0.1:
        print("   ✅ EXCELLENT accuracy (WER < 10%)")
    elif wer < 0.2:
        print("   ✅ GOOD accuracy (WER < 20%)")
    else:
        print("   ⚠️  FAIR accuracy (WER > 20%)")

In [ ]:
# Cell 9: Save Results
def save_results(results_dict, filename="lfm_complete_results.json"):
    """Save test results to JSON file."""
    results_path = Path("results")
    results_path.mkdir(exist_ok=True)

    output_file = results_path / filename
    with open(output_file, "w") as f:
        json.dump(results_dict, f, indent=2)

    print(f"✅ Results saved: {output_file}")
    return output_file


# Compile comprehensive results
results = {
    "test_info": {
        "model": HF_REPO,
        "device": device,
        "timestamp": datetime.now().isoformat(),
        "python_version": sys.version.split()[0],
    },
    "performance": {
        "model_load_time": load_time,
        "asr_latency": latency,
        "real_time_factor": real_time_factor,
        "memory_mb": memory_info.rss / 1e6,
    },
    "quality": {
        "transcription": transcribed_text if "transcribed_text" in locals() else "",
        "ground_truth": ground_truth if "ground_truth" in locals() else "",
        "wer": wer if "wer" in locals() else None,
    },
    "capabilities_tested": {
        "asr": True,
        "tts": "tts_latency" in locals(),
        "conversation": len(audio_out) > 0 if "audio_out" in locals() else False,
    },
    "audio_info": {
        "file_tested": str(audio_path) if audio_path.exists() else "dummy",
        "duration_seconds": waveform.shape[1] / sr,
        "sample_rate": sr,
    },
}

# Save results
save_results(results)

print("\n🎉 COMPLETE LFM TEST SUCCESSFUL!")
print("📋 All capabilities tested and documented")
print("🚀 Ready for systematic model comparison")